In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from RK4 import dT_dt # importe a função do seu script RK4.py

# -----------------------------
# Solução Analítica
# -----------------------------
T_amb = 25
T0 = 70
k = 0.005

t = np.linspace(0, 2000, 100)
T_analitica = T_amb + (T0 - T_amb) * np.exp(-k * t)

# -----------------------------
# RK4
def dt_dt(T, t, k, T_amb):
    return -k * (T - T_amb)

dt = 20
t_rk4 = np.arange(0, 2000+dt, dt)
T_rk4 = np.zeros(len(t_rk4))
T_rk4[0] = T0

for i in range(1, len(t_rk4)):
    ti = t_rk4[i-1]
    Ti = T_rk4[i-1]
    
    k1 = dT_dt(Ti, ti, k, T_amb)
    k2 = dT_dt(Ti + 0.5 * dt * k1, ti + 0.5 * dt, k, T_amb)
    k3 = dT_dt(Ti + 0.5 * dt * k2, ti + 0.5 * dt, k, T_amb)
    k4 = dT_dt(Ti + dt * k3, ti + dt, k, T_amb)
    
    T_rk4[i] = Ti + (dt/6)*(k1 + 2*k2 + 2*k3 + k4)

# -----------------------------
# PINN
# -----------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
t_torch = torch.tensor(t.reshape(-1,1), dtype=torch.float32, requires_grad=True).to(device)

# Carregar modelo treinado do PINN.py ou treinar novamente aqui se preferir
from PINN import PINN
model = PINN().to(device)
model.load_state_dict(torch.load('pinn_model.pth'))  # supondo que salvou o modelo treinado
model.eval()
T_pinn = model(t_torch).cpu().detach().numpy()

# -----------------------------
# Plot
# -----------------------------
plt.plot(t, T_analitica, label='Analítica')
plt.plot(t_rk4, T_rk4, 'o-', label='RK4')
plt.plot(t, T_pinn, '--', label='PINN')
plt.xlabel('Tempo (s)')
plt.ylabel('Temperatura (°C)')
plt.title('Comparação: Analítica vs RK4 vs PINN')
plt.legend()
plt.grid(True)
plt.show()


ModuleNotFoundError: No module named 'RK4'